In [2]:
import requests
import csv
from bs4 import BeautifulSoup
import time
import re
import pandas as pd

In [ ]:
url = "https://www.bcn.cl/siit/mapoteca/comunas"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Language": "es-CL,es;q=0.9,en;q=0.8",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "Cache-Control": "max-age=0"
}

In [4]:
page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.text, 'html.parser')

region_header = soup.find('a', string="Región Metropolitana de Santiago")

accordion_group = region_header.find_parent('div', class_='accordion-group')

comuna_links = accordion_group.find_all('a', class_='list-group-item')

comunas_urls = []

for link in comuna_links:
    nombre = link.text.strip()
    onclick = link.get('onclick', '')

    match = re.search(r"envia3\('comuna_view',\s*'.*?',\s*'(\d+)'\)", onclick)
    if match:
        codigo = match.group(1)
        url = f"https://www.bcn.cl/siit/reportescomunales/comunas_v.html?anno=2025&idcom={codigo}"
        comunas_urls.append((nombre, url))


In [5]:
df_comunas = pd.DataFrame(comunas_urls, columns=['comuna', 'url'])
print(df_comunas['url'])

0     https://www.bcn.cl/siit/reportescomunales/comu...
1     https://www.bcn.cl/siit/reportescomunales/comu...
2     https://www.bcn.cl/siit/reportescomunales/comu...
3     https://www.bcn.cl/siit/reportescomunales/comu...
4     https://www.bcn.cl/siit/reportescomunales/comu...
5     https://www.bcn.cl/siit/reportescomunales/comu...
6     https://www.bcn.cl/siit/reportescomunales/comu...
7     https://www.bcn.cl/siit/reportescomunales/comu...
8     https://www.bcn.cl/siit/reportescomunales/comu...
9     https://www.bcn.cl/siit/reportescomunales/comu...
10    https://www.bcn.cl/siit/reportescomunales/comu...
11    https://www.bcn.cl/siit/reportescomunales/comu...
12    https://www.bcn.cl/siit/reportescomunales/comu...
13    https://www.bcn.cl/siit/reportescomunales/comu...
14    https://www.bcn.cl/siit/reportescomunales/comu...
15    https://www.bcn.cl/siit/reportescomunales/comu...
16    https://www.bcn.cl/siit/reportescomunales/comu...
17    https://www.bcn.cl/siit/reportescomunales/

In [ ]:
def limpiar_valor(valor):
    if valor is None or valor == "":
        return valor
    valor_str = str(valor)
    if ',' in valor_str:
        valor_str = valor_str.replace('.', '')
        valor_str = valor_str.replace(',', '.')
    else:
        partes_punto = valor_str.split('.')
        if len(partes_punto) > 2 or (len(partes_punto) == 2 and len(partes_punto[-1]) > 2):
            valor_str = valor_str.replace('.', '')
    valor_str = valor_str.replace('%', '')
    valor_str = valor_str.strip()
    return valor_str

def extraer_dato_comuna(filas, posicion_comuna=0):
    for fila in filas[1:]:
        celdas = fila.find_all(['td', 'th'])
        if len(celdas) > posicion_comuna:
            texto_celdas = [celda.get_text(strip=True).replace("\xa0", " ") for celda in celdas]
            if "Comuna de" in texto_celdas[0]:
                return texto_celdas
    return None

datos_todas_comunas = []

for index, row in df_comunas.iterrows():
    comuna = row['comuna']
    url = row['url']

    print(f"\n📍 Procesando {comuna}...")

    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    bloques = soup.find_all("div", class_="z-depth-1 p-4 rounded mb-5")

    print(f"🔍 Se encontraron {len(bloques)} bloques con esa clase.")

    datos_comuna = {'Comuna': comuna}

    for i, bloque in enumerate(bloques, start=1):
        h6 = bloque.find("h6")
        if h6:
            h6_text = h6.get_text(strip=True)

            tabla = bloque.find("table")
            if tabla:
                filas = tabla.find_all("tr")

                if "1.1" in h6_text and "Población total" in h6_text:
                    datos_fila = extraer_dato_comuna(filas)
                    if datos_fila and len(datos_fila) > 2:
                        datos_comuna['Población 2024'] = limpiar_valor(datos_fila[2])

                elif "1.2" in h6_text and ("Urbana-Rural" in h6_text or "ÁreaUrbana-Rural" in h6_text):
                    datos_fila = extraer_dato_comuna(filas)
                    if datos_fila and len(datos_fila) > 6:
                        datos_comuna['Ruralidad 2024 (%)'] = limpiar_valor(datos_fila[6])

                elif "1.3" in h6_text and "sexo" in h6_text:
                    datos_fila = extraer_dato_comuna(filas)
                    if datos_fila and len(datos_fila) > 6:
                        datos_comuna['Índice Masculinidad 2024'] = limpiar_valor(datos_fila[6])

                elif "1.4" in h6_text and "grupos de edad" in h6_text:
                    for fila in filas[1:]:
                        celdas = fila.find_all(['td', 'th'])
                        if len(celdas) >= 4:
                            texto_celdas = [celda.get_text(strip=True).replace("\xa0", " ") for celda in celdas]
                            if texto_celdas[0] and texto_celdas[0] != "Total":
                                if "Censo 2017" not in texto_celdas[0]:
                                    grupo_nombre = texto_celdas[0].replace(" ", "_")
                                    datos_comuna[f'Grupo_Etario_{grupo_nombre}_2024_(%)'] = limpiar_valor(texto_celdas[3])

                elif "1.5" in h6_text and ("Dependencia" in h6_text or "Adultos Mayores" in h6_text):
                    datos_fila = extraer_dato_comuna(filas)
                    if datos_fila and len(datos_fila) > 4:
                        datos_comuna['IDD 2024'] = limpiar_valor(datos_fila[2])
                        datos_comuna['IAM 2024'] = limpiar_valor(datos_fila[4])

                elif "2.1" in h6_text and "Pobreza por ingresos" in h6_text:
                    datos_fila = extraer_dato_comuna(filas)
                    if datos_fila and len(datos_fila) > 2:
                        datos_comuna['Pobreza por ingresos 2022 (%)'] = limpiar_valor(datos_fila[2])

                elif "2.2" in h6_text and "multidimensional" in h6_text:
                    datos_fila = extraer_dato_comuna(filas)
                    if datos_fila and len(datos_fila) > 2:
                        datos_comuna['Pobreza multidimensional 2022 (%)'] = limpiar_valor(datos_fila[2])

                elif "2.3" in h6_text and ("indígenas" in h6_text or "Extranjeros" in h6_text):
                    datos_fila = extraer_dato_comuna(filas)
                    if datos_fila and len(datos_fila) > 2:
                        datos_comuna['Pueblos indígenas 2025 (%)'] = limpiar_valor(datos_fila[1])
                        datos_comuna['Extranjeros 2025 (%)'] = limpiar_valor(datos_fila[2])

                elif "2.4" in h6_text and ("servicios básicos" in h6_text or "hacinados" in h6_text):
                    datos_fila = extraer_dato_comuna(filas)
                    if datos_fila and len(datos_fila) > 2:
                        datos_comuna['Carentes servicios básicos 2025 (%)'] = limpiar_valor(datos_fila[1])
                        datos_comuna['Hogares hacinados 2025 (%)'] = limpiar_valor(datos_fila[2])

                elif "3.1" in h6_text and "salud" in h6_text:
                    for fila in filas[2:-1]:
                        celdas = fila.find_all(['td', 'th'])
                        if len(celdas) >= 2:
                            texto_celdas = [celda.get_text(strip=True).replace("\xa0", " ") for celda in celdas]
                            tipo_establecimiento = texto_celdas[0]
                            comuna_valor = limpiar_valor(texto_celdas[1])
                            datos_comuna[f"Cantidad de: {tipo_establecimiento} 2025"] = comuna_valor

                elif "3.2" in h6_text and "Población inscrita" in h6_text:
                    datos_fila = extraer_dato_comuna(filas)
                    if datos_fila and len(datos_fila) >= 3:
                        datos_comuna['Población inscrita comuna 2023'] = datos_fila[0]

                elif "3.3" in h6_text and ("fecundidad" in h6_text.lower() or "natalidad" in h6_text.lower()):
                    datos_fila = extraer_dato_comuna(filas)
                    if datos_fila and len(datos_fila) > 2:
                        datos_comuna['Fecundidad 2022'] = limpiar_valor(datos_fila[1])
                        datos_comuna['Natalidad 2022'] = limpiar_valor(datos_fila[2])

                elif "3.4" in h6_text and ("mortalidad general" in h6_text.lower() or "mortalidad infantil" in h6_text.lower()):
                    datos_fila = extraer_dato_comuna(filas)
                    if datos_fila and len(datos_fila) > 2:
                        datos_comuna['Mortalidad General 2022 (c/1.000 hab)'] = limpiar_valor(datos_fila[1])
                        datos_comuna['Mortalidad Infantil 2022 (c/1.000 nac.vivos)'] = limpiar_valor(datos_fila[2])

                elif "4.2" in h6_text and "Matrícula escolar" in h6_text:
                    for fila in filas[1:]:
                        celdas = fila.find_all(['td', 'th'])
                        if len(celdas) >= 3:
                            texto_celdas = [celda.get_text(strip=True).replace("\xa0", " ") for celda in celdas]
                            tipo_matricula = texto_celdas[0]
                            if "Matricula Municipal" in tipo_matricula:
                                datos_comuna['Matrícula Municipal 2024'] = limpiar_valor(texto_celdas[2])
                            elif "Matricula Subvencionada" in tipo_matricula:
                                datos_comuna['Matrícula Subvencionada 2024'] = limpiar_valor(texto_celdas[2])
                            elif "Matricula Particular Pagado" in tipo_matricula:
                                datos_comuna['Matrícula Particular Pagado 2024'] = limpiar_valor(texto_celdas[2])
                            elif "Matricula Corporación de Administración Delegada" in tipo_matricula:
                                datos_comuna['Matrícula Corporación Admin Delegada 2024'] = limpiar_valor(texto_celdas[2])
                            elif "Matricula Servicio Local de Educación" in tipo_matricula:
                                datos_comuna['Matrícula Servicio Local Educación 2024'] = limpiar_valor(texto_celdas[2])

                elif "4.3" in h6_text and "nivel de enseñanza" in h6_text:
                    for fila in filas[2:]:
                        celdas = fila.find_all(['td', 'th'])
                        if len(celdas) >= 3:
                            texto_celdas = [celda.get_text(strip=True).replace("\xa0", " ") for celda in celdas]
                            nivel_enseñanza = texto_celdas[0]
                            if nivel_enseñanza and nivel_enseñanza != "Total":
                                if len(texto_celdas) > 2:
                                    valor_comuna_2024 = limpiar_valor(texto_celdas[2])
                                    if valor_comuna_2024 == "-":
                                        valor_comuna_2024 = "0"
                                    nivel_limpio = nivel_enseñanza.replace(",", "").replace(" ", "_")
                                    clave_columna = f"Matrícula_{nivel_limpio}_2024"
                                    datos_comuna[clave_columna] = valor_comuna_2024

                elif "4.4" in h6_text and "4to básico" in h6_text:
                    datos_fila = extraer_dato_comuna(filas)
                    if datos_fila and len(datos_fila) > 2:
                        datos_comuna['SIMCE 4to Básico Lectura 2022'] = limpiar_valor(datos_fila[1])
                        datos_comuna['SIMCE 4to Básico Matemáticas 2022'] = limpiar_valor(datos_fila[2])

                elif "4.5" in h6_text and "2do Medio" in h6_text:
                    datos_fila = extraer_dato_comuna(filas)
                    if datos_fila and len(datos_fila) > 2:
                        datos_comuna['SIMCE 2do Medio Lectura 2022'] = limpiar_valor(datos_fila[1])
                        datos_comuna['SIMCE 2do Medio Matemáticas 2022'] = limpiar_valor(datos_fila[2])

                elif "5.1" in h6_text and ("empresas" in h6_text.lower() or "tamaño" in h6_text.lower()):
                    for fila in filas[2:-1]:
                        celdas = fila.find_all(['td', 'th'])
                        if len(celdas) >= 4:
                            texto_celdas = [celda.get_text(strip=True).replace("\xa0", " ") for celda in celdas]
                            tramo_empresa = texto_celdas[0]
                            if tramo_empresa:
                                if len(texto_celdas) > 3:
                                    valor_comuna_2023 = limpiar_valor(texto_celdas[3])
                                    if tramo_empresa == "Sin Ventas/Sin Información":
                                        clave_columna = "Empresas_Sin_Ventas_Sin_Información_2023"
                                    else:
                                        tramo_limpio = tramo_empresa.replace(" ", "_")
                                        clave_columna = f"Empresas_{tramo_limpio}_2023"
                                    datos_comuna[clave_columna] = valor_comuna_2023

                elif "5.2" in h6_text and ("trabajadores" in h6_text.lower() or "dependientes" in h6_text.lower()):
                    for fila in filas[2:-1]:
                        celdas = fila.find_all(['td', 'th'])
                        if len(celdas) >= 4:
                            texto_celdas = [celda.get_text(strip=True).replace("\xa0", " ") for celda in celdas]
                            tramo_empresa = texto_celdas[0]
                            if tramo_empresa:
                                if len(texto_celdas) > 3:
                                    valor_comuna_2023 = limpiar_valor(texto_celdas[3])
                                    if tramo_empresa == "Sin Ventas/Sin Información":
                                        clave_columna = "Trabajadores_Sin_Ventas_Sin_Información_2023"
                                    else:
                                        tramo_limpio = tramo_empresa.replace(" ", "_")
                                        clave_columna = f"Trabajadores_{tramo_limpio}_2023"
                                    datos_comuna[clave_columna] = valor_comuna_2023

                elif "5.3" in h6_text and ("rubro" in h6_text.lower() or "económico" in h6_text.lower()):
                    for fila in filas[2:]:
                        celdas = fila.find_all(['td', 'th'])
                        if len(celdas) >= 4:
                            texto_celdas = [celda.get_text(strip=True).replace("\xa0", " ") for celda in celdas]
                            rubro_economico = texto_celdas[0]
                            if rubro_economico:
                                if len(texto_celdas) > 3:
                                    valor_comuna_2023 = limpiar_valor(texto_celdas[3])
                                    if "Agricultura ganadería silvicultura y pesca" in rubro_economico:
                                        clave_columna = "Empresas_Agricultura_Ganadería_2023"
                                    elif "Explotación de minas y canteras" in rubro_economico:
                                        clave_columna = "Empresas_Minería_2023"
                                    elif "Industria manufacturera" in rubro_economico:
                                        clave_columna = "Empresas_Industria_Manufacturera_2023"
                                    elif "Suministro de electricidad gas vapor" in rubro_economico:
                                        clave_columna = "Empresas_Suministro_Electricidad_2023"
                                    elif "Suministro de agua evacuación" in rubro_economico:
                                        clave_columna = "Empresas_Suministro_Agua_2023"
                                    elif "Construcción" == rubro_economico:
                                        clave_columna = "Empresas_Construcción_2023"
                                    elif "Comercio al por mayor y al por menor" in rubro_economico:
                                        clave_columna = "Empresas_Comercio_2023"
                                    elif "Transporte y almacenamiento" in rubro_economico:
                                        clave_columna = "Empresas_Transporte_2023"
                                    elif "Actividades de alojamiento y de servicio de comidas" in rubro_economico:
                                        clave_columna = "Empresas_Alojamiento_Comidas_2023"
                                    elif "Información y comunicaciones" in rubro_economico:
                                        clave_columna = "Empresas_Información_Comunicaciones_2023"
                                    elif "Actividades financieras y de seguros" in rubro_economico:
                                        clave_columna = "Empresas_Financieras_Seguros_2023"
                                    elif "Actividades inmobiliarias" in rubro_economico:
                                        clave_columna = "Empresas_Inmobiliarias_2023"
                                    elif "Actividades profesionales científicas y técnicas" in rubro_economico:
                                        clave_columna = "Empresas_Profesionales_Científicas_2023"
                                    elif "Actividades de servicios administrativos" in rubro_economico:
                                        clave_columna = "Empresas_Servicios_Administrativos_2023"
                                    elif "Administración pública y defensa" in rubro_economico:
                                        clave_columna = "Empresas_Administración_Pública_2023"
                                    elif "Enseñanza" == rubro_economico:
                                        clave_columna = "Empresas_Enseñanza_2023"
                                    elif "Actividades de atención de la salud" in rubro_economico:
                                        clave_columna = "Empresas_Salud_2023"
                                    elif "Actividades artísticas de entretenimiento" in rubro_economico:
                                        clave_columna = "Empresas_Artísticas_Entretenimiento_2023"
                                    elif "Otras actividades de servicios" in rubro_economico:
                                        clave_columna = "Empresas_Otras_Actividades_2023"
                                    elif "Actividades de los hogares como empleadores" in rubro_economico:
                                        clave_columna = "Empresas_Hogares_Empleadores_2023"
                                    elif "Actividades de organizaciones y órganos extraterritoriales" in rubro_economico:
                                        clave_columna = "Empresas_Organizaciones_Extraterritoriales_2023"
                                    elif "Sin información" in rubro_economico:
                                        clave_columna = "Empresas_Sin_Información_2023"
                                    else:
                                        rubro_limpio = rubro_economico.replace(" ", "_").replace(",", "")[:50]
                                        clave_columna = f"Empresas_{rubro_limpio}_2023"
                                    datos_comuna[clave_columna] = valor_comuna_2023

                elif "5.4" in h6_text and ("trabajadores" in h6_text.lower() and "rubro" in h6_text.lower()):
                    for fila in filas[2:]:
                        celdas = fila.find_all(['td', 'th'])
                        if len(celdas) >= 4:
                            texto_celdas = [celda.get_text(strip=True).replace("\xa0", " ") for celda in celdas]
                            rubro_economico = texto_celdas[0]
                            if rubro_economico:
                                if len(texto_celdas) > 3:
                                    valor_comuna_2023 = limpiar_valor(texto_celdas[3])
                                    if "Agricultura ganadería silvicultura y pesca" in rubro_economico:
                                        clave_columna = "Trabajadores_Agricultura_Ganadería_2023"
                                    elif "Explotación de minas y canteras" in rubro_economico:
                                        clave_columna = "Trabajadores_Minería_2023"
                                    elif "Industria manufacturera" in rubro_economico:
                                        clave_columna = "Trabajadores_Industria_Manufacturera_2023"
                                    elif "Suministro de electricidad gas vapor" in rubro_economico:
                                        clave_columna = "Trabajadores_Suministro_Electricidad_2023"
                                    elif "Suministro de agua evacuación" in rubro_economico:
                                        clave_columna = "Trabajadores_Suministro_Agua_2023"
                                    elif "Construcción" == rubro_economico:
                                        clave_columna = "Trabajadores_Construcción_2023"
                                    elif "Comercio al por mayor y al por menor" in rubro_economico:
                                        clave_columna = "Trabajadores_Comercio_2023"
                                    elif "Transporte y almacenamiento" in rubro_economico:
                                        clave_columna = "Trabajadores_Transporte_2023"
                                    elif "Actividades de alojamiento y de servicio de comidas" in rubro_economico:
                                        clave_columna = "Trabajadores_Alojamiento_Comidas_2023"
                                    elif "Información y comunicaciones" in rubro_economico:
                                        clave_columna = "Trabajadores_Información_Comunicaciones_2023"
                                    elif "Actividades financieras y de seguros" in rubro_economico:
                                        clave_columna = "Trabajadores_Financieras_Seguros_2023"
                                    elif "Actividades inmobiliarias" in rubro_economico:
                                        clave_columna = "Trabajadores_Inmobiliarias_2023"
                                    elif "Actividades profesionales científicas y técnicas" in rubro_economico:
                                        clave_columna = "Trabajadores_Profesionales_Científicas_2023"
                                    elif "Actividades de servicios administrativos" in rubro_economico:
                                        clave_columna = "Trabajadores_Servicios_Administrativos_2023"
                                    elif "Administración pública y defensa" in rubro_economico:
                                        clave_columna = "Trabajadores_Administración_Pública_2023"
                                    elif "Enseñanza" == rubro_economico:
                                        clave_columna = "Trabajadores_Enseñanza_2023"
                                    elif "Actividades de atención de la salud" in rubro_economico:
                                        clave_columna = "Trabajadores_Salud_2023"
                                    elif "Actividades artísticas de entretenimiento" in rubro_economico:
                                        clave_columna = "Trabajadores_Artísticas_Entretenimiento_2023"
                                    elif "Otras actividades de servicios" in rubro_economico:
                                        clave_columna = "Trabajadores_Otras_Actividades_2023"
                                    elif "Actividades de los hogares como empleadores" in rubro_economico:
                                        clave_columna = "Trabajadores_Hogares_Empleadores_2023"
                                    elif "Actividades de organizaciones y órganos extraterritoriales" in rubro_economico:
                                        clave_columna = "Trabajadores_Organizaciones_Extraterritoriales_2023"
                                    elif "Sin información" in rubro_economico:
                                        clave_columna = "Trabajadores_Sin_Información_2023"
                                    else:
                                        rubro_limpio = rubro_economico.replace(" ", "_").replace(",", "")[:50]
                                        clave_columna = f"Trabajadores_{rubro_limpio}_2023"
                                    datos_comuna[clave_columna] = valor_comuna_2023

                elif "6.1" in h6_text and ("Ingresos Propios" in h6_text or "ingreso" in h6_text.lower()):
                    for fila in filas[1:]:
                        celdas = fila.find_all(['td', 'th'])
                        if len(celdas) >= 3:
                            texto_celdas = [celda.get_text(strip=True).replace("\xa0", " ") for celda in celdas]
                            tipo_ingreso = texto_celdas[0]
                            if "Disponibilidad Presupuestaria Municipal por Habitante" in tipo_ingreso:
                                if len(texto_celdas) > 2:
                                    valor_2023 = limpiar_valor(texto_celdas[2])
                                    datos_comuna['Disponibilidad_Presupuestaria_por_Habitante_2023_(M$)'] = valor_2023
                                break

                elif "7.1" in h6_text and ("Delitos de Mayor Connotación Social" in h6_text or "Denuncias" in h6_text):
                    datos_fila = extraer_dato_comuna(filas)
                    if datos_fila and len(datos_fila) > 3:
                        datos_comuna['Delitos Mayor Connotación Social 2023 (c/100.000 hab)'] = limpiar_valor(datos_fila[3])

                elif "7.2" in h6_text and "Violencia Intrafamiliar" in h6_text:
                    datos_fila = extraer_dato_comuna(filas)
                    if datos_fila and len(datos_fila) > 3:
                        datos_comuna['Violencia Intrafamiliar 2023 (c/100.000 hab)'] = limpiar_valor(datos_fila[3])

    datos_todas_comunas.append(datos_comuna)

    print(f"✅ Datos extraídos para {comuna}")

print("\n📝 Creando archivo CSV...")
df_resultados = pd.DataFrame(datos_todas_comunas)

nombre_archivo = "../../data/external/scraped/datos_comunas_extraidos.csv"
df_resultados.to_csv(nombre_archivo, index=False, encoding='utf-8-sig')

print(f"✅ Archivo '{nombre_archivo}' creado exitosamente!")
print(f"📊 Total de comunas procesadas: {len(datos_todas_comunas)}")
print(f"📋 Total de columnas en el CSV: {len(df_resultados.columns)}")

print("\n👀 Vista previa del archivo CSV:")
print(df_resultados.head())


📍 Procesando Alhué...
🔍 Se encontraron 33 bloques con esa clase.
✅ Datos extraídos para Alhué

📍 Procesando Buin...
🔍 Se encontraron 33 bloques con esa clase.
✅ Datos extraídos para Buin

📍 Procesando Calera de Tango...
🔍 Se encontraron 33 bloques con esa clase.
✅ Datos extraídos para Calera de Tango

📍 Procesando Cerrillos...
🔍 Se encontraron 33 bloques con esa clase.
✅ Datos extraídos para Cerrillos

📍 Procesando Cerro Navia...
🔍 Se encontraron 33 bloques con esa clase.
✅ Datos extraídos para Cerro Navia

📍 Procesando Colina...
🔍 Se encontraron 33 bloques con esa clase.
✅ Datos extraídos para Colina

📍 Procesando Conchalí...
🔍 Se encontraron 33 bloques con esa clase.
✅ Datos extraídos para Conchalí

📍 Procesando Curacaví...
🔍 Se encontraron 33 bloques con esa clase.
✅ Datos extraídos para Curacaví

📍 Procesando El Bosque...
🔍 Se encontraron 33 bloques con esa clase.
✅ Datos extraídos para El Bosque

📍 Procesando El Monte...
🔍 Se encontraron 33 bloques con esa clase.
✅ Datos extraído

In [8]:
df_resultados.head()

,Comuna,Población 2024,Índice Masculinidad 2024,Grupo_Etario_0_a_14_2024_(%),Grupo_Etario_15_a_29_2024_(%),Grupo_Etario_30_a_44_2024_(%),Grupo_Etario_45_a_64_2024_(%),Grupo_Etario_65_o_mas_2024_(%),IDD 2024,IAM 2024,...,Trabajadores_Enseñanza_2023,Trabajadores_Salud_2023,Trabajadores_Artísticas_Entretenimiento_2023,Trabajadores_Otras_Actividades_2023,Trabajadores_Hogares_Empleadores_2023,Trabajadores_Organizaciones_Extraterritoriales_2023,Trabajadores_Sin_Información_2023,Disponibilidad_Presupuestaria_por_Habitante_2023_(M$),Delitos Mayor Connotación Social 2023 (c/100.000 hab),Violencia Intrafamiliar 2023 (c/100.000 hab)
0,Alhué,7768,98.9,20.7,20,23.3,23,13,51,62.8,...,0,5,0,11,0,0,0,2724,1947.6,929.2
1,Buin,116969,93.6,21.3,19.6,26.5,22.2,10.4,46.4,48.7,...,1614,1788,602,433,0,0,10,341,1982.5,459.4
2,Calera de Tango,25491,97.9,18,21.1,20.7,26.7,13.5,46,74.8,...,492,70,10,778,0,0,4,487,2044.1,548.2
3,Cerrillos,85041,94,18.7,21.7,22.9,23.7,13,46.3,69.7,...,947,1179,39,645,0,0,31,335,3434.2,682.0
4,Cerro Navia,127250,96,17.6,21.6,20.6,25.4,14.7,47.7,83.8,...,1312,4044,25,1210,0,0,1,286,3053.7,675.3
